In [1]:
import openai
import requests
import PyPDF2
import re
import os
import requests
import pandas as pd
import tiktoken
import time
from openai import OpenAI
from io import StringIO
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import numpy as np
import ast

In [2]:
client = OpenAI(
     api_key=os.environ.get("OPENAI_API_KEY"),)

In [3]:
def model_1(df):
    """Model 1 will turn text in dataframe to a summarized reaction condition table.The dataframe should have a column "file name" and a column "exp content"."""
    response_msgs = []

    for index, row in df.iterrows():
        column1_value = row[df.columns[0]]
        column2_value = row['content']

        max_tokens = 3000
        if count_tokens(column2_value) > max_tokens:
            context_list = split_content(column2_value, max_tokens)
        else:
            context_list = [column2_value]

        answers = ''  # Collect answers from chatGPT
        for context in context_list:
            print("Start to analyze paper " + str(column1_value) )
            user_heading = f"This is an experimental section on MOF synthesis from paper {column1_value}\n\nContext:\n{context}"
            user_ending = """Q: Can you summarize the following details in a table: 
            compound name or chemical formula (if the name is not provided), metal source, metal amount, organic linker(s), 
            linker amount, modulator, modulator amount or volume, solvent(s), solvent volume(s), reaction temperature, 
            and reaction time? If any information is not provided or you are unsure, use "N/A". 
            Please focus on extracting experimental conditions from only the MOF synthesis and ignore information related to organic linker synthesis, 
            MOF postsynthetic modification, high throughput (HT) experiment details or catalysis reactions. 
            If multiple conditions are provided for the same compound, use multiple rows to represent them. If multiple units or components are provided for the same factor (e.g.  g and mol for the weight, multiple linker or metals, multiple temperature and reaction time, mixed solvents, etc), include them in the same cell and separate by comma.
            The table should have 11 columns, all in lowercase:
            | compound name | metal source | metal amount | linker | linker amount | modulator | modulator amount or volume | solvent | solvent volume | reaction temperature | reaction time |

            A:"""   

            attempts = 3
            while attempts > 0:
                try:
                    response = client.chat.completions.create(
                        model='gpt-3.5-turbo',
                        messages=[{
                            "role": "system",
                            "content": """Answer the question as truthfully as possible using the provided context,
                                        and if the answer is not contained within the text below, say "N/A" """
                        },
                            {"role": "user", "content": user_heading + user_ending}]
                    )
                    answer_str = response.choices[0].message.content
                    if not answer_str.lower().startswith("n/a"):
                        answers += '\n' + answer_str
                    break
                except Exception as e:
                    attempts -= 1
                    if attempts <= 0:
                        print(f"Error: Failed to process paper {column1_value}. Skipping. (model 1)")
                        break
                    print(f"Error: {str(e)}. Retrying in 60 seconds. {attempts} attempts remaining. (model 1)")
                    time.sleep(60)

        response_msgs.append(answers)
    df = df.copy()
    df.loc[:, 'summarized'] = response_msgs
    return df


In [4]:
def tabulate_condition(df,column_name):
    """This function converts the text from a ChatGPT conversation into a DataFrame.
    It also cleans the DataFrame by dropping additional headers and empty lines.    """
    
    table_text = df[column_name].str.cat(sep='\n')

    # Remove leading and trailing whitespace
    table_text = table_text.strip()
    
    # Split the table into rows
    rows = table_text.split('\n')

    # Extract the header row and the divider row
    header_row, divider_row, *data_rows = rows

    # Extract column names from the header row

    column_names = ['compound name', 'metal source', 'metal amount', 'linker', 'linker amount',
                   'modulator', 'modulator amount or volume', 'solvent', 'solvent volume', 'reaction temperature',
                   'reaction time']

    # Create a list of dictionaries to store the table data
    data = []

    # Process each data row
    for row in data_rows:

        # Split the row into columns
        columns = [col.strip() for col in row.split('|') if col.strip()]
    
        # Create a dictionary to store the row data
        row_data = {col_name: col_value for col_name, col_value in zip(column_names, columns)}
    
        # Append the dictionary to the data list
        data.append(row_data)
        
    df = pd.DataFrame(data)

    
    """Make df clean by drop additional header and empty lines """
    def contains_pattern(s, patterns):
        return any(re.search(p, s) for p in patterns)

    def drop_rows_with_patterns(df, column_name):
        #empty cells, N/A cells and header cells
        patterns = [r'^\s*$', r'--',r'-\s-', r'compound', r'Compound',r'Compound name', r'Compound Name',
                r'NaN',r'N/A',r'n/a',r'\nN/A', r'note', r'Note']
        
        mask = df[column_name].apply(lambda x: not contains_pattern(str(x), patterns))
        filtered_df = df[mask]
    
        return filtered_df

In [10]:
def count_tokens(text):
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(text))
    return num_tokens

In [15]:
syn_df = pd.read_csv("C:/Users/89751/OneDrive/桌面/228paper_info1.csv")

In [16]:
syn_df

,file ID,file name short,doi,content
0,1,05acetylene,10.1038/nature03852,Synthesis of Cu2(pzdc)2(pyz)·2H2O (hydrated co...
1,2,05RodPackings,10.1021/ja045123o,"""Synthesis of Compounds. Zn3(OH)2(BPDC)2â(DEF)..."


In [20]:
model_1_table = tabulate_condition(model_1(syn_df),"summarized")

Start to analyze paper 1
Start to analyze paper 2


In [21]:
print(model_1_table)

None
